In [151]:
import cv2
from skimage.transform import resize
from os import listdir, walk, makedirs, replace
from os.path import isfile, join
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
import pickle
import itertools
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam



In [ ]:
test0 = pd.read_csv('data/Data/fungiclef2022/FungiCLEF2022_test_metadata.csv',sep = ',')

In [ ]:
test0.head(3)

# START OF DATA MIX BETWEEN BIG AND SMALL DATA SET
- ATTENTION the lines below do not work systimatically but have worke once and now I keep them as a 
reminder

In [223]:
#  Gives a list of all the files in the directory with the highest photographed mashroom.
# TODO: make folders of the top20 most frequent mushrooms and put them in data folders.

train_BigData = pd.read_csv('data/Data/fungiclef2022/DF20-train_metadata.csv',sep = ',')
# find the 20 categories with the most pictures stored
train_BigData['counts'] = 1
train_BigData.groupby(['scientificName']).sum().nlargest(20, 'counts')
new_fungi = train_BigData.groupby(['scientificName']).sum().nlargest(20, 'counts')

fungi = new_fungi[["counts"]]
fungi = fungi.reset_index(level=0)


In [ ]:
k=20
G = train_BigData.loc[train_BigData['scientificName'] == fungi.scientificName[k]]
print(fungi.scientificName[k])
g = G.image_path.tolist()



In [220]:

# make directory
name ='Meripilus_giganteus'
os.makedirs( 'data/Data/Mushrooms/'+name)

directory = 'data/Data/fungiclef2022/DF20-300px_train/DF20_300/' 
image_list = []
for ls in G.image_path.tolist():
    image_list.append(directory + ls)
    




In [221]:
# TODO: move images. Mix BIG with SMALL KAggle data set to result in a total of (29 species)

def move_image(name):
    """
    Read files from directory
    """
    directory = 'data/Data/Mushrooms/'+name+'/' 

    for idx, ls in enumerate(image_list):
        # print(ls)
        # print(directory+ g[idx])
        os.replace(ls, directory+ g[idx])

move_image(name)


# END OF DATA MIX BETWEEN BIG AND SMALL DATA SET

In [ ]:
# TODO split data train valid test (75% train, 15% valid, 10% test)

# TODO: augmentation for the mushrooms with small number of images in train and valid

# TODO: downsize each image per folder and pickle a banch per kind of mushroom

# TODO: unpickle and connect

# TODO: model

- During night we attempt to downsize the images from Big Data from the train data while keeping the labels

In [ ]:
directory = 'data/Data/fungiclef2022/DF20-300px_train/DF20_300/' 
train_BigData['full_path_image'] = directory + train_BigData.image_path

In [ ]:
# make a list of labels and directories
BIG_DATA_train = train_BigData.filter(['scientificName','image_path', 'full_path_image'], axis=1)
#downsize_in_batches()

In [ ]:
BIG_DATA_train.full_path_image

In [ ]:
BIG_DATA_train.scientificName

In [ ]:
def image_reshape_BD(dir, batch, new_size = (224,224),k = 300, gray =False):
    """
    works for Data from csv and random labels.
    give the list of photos with the correct pixel size you want to 
    downsize. If you want Gray then write True.
    """
    items_resized = []
    label = []
    for idx, dir in enumerate(dir[(batch-1)*k:batch * k],(batch-1)*k):     
        item = cv2.imread(dir)
        label.append(idx)
        if gray == True:
            item_gray = cv2.cvtColor(item, cv2.COLOR_BGR2GRAY)
            item_resized_gray = resize(item_gray, new_size)
            items_resized.append(item_resized_gray)
        else:
            item_resized_colored = resize(item, new_size)
            items_resized.append(item_resized_colored)
    direct = 'data/Data/fungiclef2022/DF20-300px_train/' 
    with open(direct + "downsized/mushrooms_resized_part_"+ str(batch), "wb") as fp:   #Pickling
        pickle.dump(items_resized, fp)
    return items_resized, label
    
def downsize_in_batches(directories):
    """
    This function downsizes the images in batches of 300pics.
    directories = list of the directories of the images.
    """
    batch = 1
    for idx, dir in enumerate(directories):
        if idx % 300 == 0:
            print(batch)
            image_reshape_BD(directories,batch)
            batch += 1 


In [ ]:
# make a list of labels and directories
downsize_in_batches(BIG_DATA_train.full_path_image)

- END

- During night we attempt to downsize the images from Big Data from the valid data while keeping the labels

In [ ]:
test_BigData = pd.read_csv('data/Data/fungiclef2022/FungiCLEF2022_test_metadata.csv',sep = ',')
test_BigData.head(2)

In [ ]:
directory = 'data/Data/fungiclef2022/DF21-images-300_val/DF21_300/' 
test_BigData['full_path_image'] = directory + test_BigData.filename

In [ ]:
# make a list of labels and directories
BIG_DATA_test = test_BigData.filter(['ObservationId','filename', 'full_path_image'], axis=1)
#downsize_in_batches()

In [ ]:
def image_reshape_BD(dir, batch, new_size = (224,224),k = 300, gray =False):
    """
    works for Data from csv and random labels.
    give the list of photos with the correct pixel size you want to 
    downsize. If you want Gray then write True.
    """
    items_resized = []
    label = []
    for idx, dir in enumerate(dir[(batch-1)*k:batch * k],(batch-1)*k):     
        item = cv2.imread(dir)
        label.append(idx)
        if gray == True:
            item_gray = cv2.cvtColor(item, cv2.COLOR_BGR2GRAY)
            item_resized_gray = resize(item_gray, new_size)
            items_resized.append(item_resized_gray)
        else:
            item_resized_colored = resize(item, new_size)
            items_resized.append(item_resized_colored)
    direct = 'data/Data/fungiclef2022/DF21-images-300_val/' 
    with open(direct + "downsized/mushrooms_resized_part_"+ str(batch), "wb") as fp:   #Pickling
        pickle.dump(items_resized, fp)
    return items_resized, label
    
def downsize_in_batches(directories):
    """
    This function downsizes the images in batches of 300pics.
    directories = list of the directories of the images.
    """
    batch = 1
    for idx, dir in enumerate(directories):
        if idx % 300 == 0:
            print(batch)
            image_reshape_BD(directories,batch)
            batch += 1 


In [ ]:
# make a list of labels and directories
downsize_in_batches(BIG_DATA_test.full_path_image)

- END

- From Big Data mushrooms we have two lists, splitted in test and validation.
- - 1604 kinds of fungis
- - 266344 / 19594 , test/ val


- Smalll data set
- - Categories : 'Agaricus' : 0,'Amanita' : 1,'Boletus' : 2,'Cortinarius' : 3,'Entoloma' : 4,'Hygrocybe' : 5,
    'Lactarius' : 6,'Russula' : 7,'Suillus' : 8
- - number of pics per category: [352, 749, 1072, 835, 363, 315, 1562, 1147, 310]

# Day-1
- Work on the small data set.
- - make batches of downsize files
- - see issue with the premature end of JPEG
- - - make model run with 300 img per category
- - - make model work for 1000 pics per category using augmentation.Keras

In [ ]:
def getList(dict):
    """
    This function returns the list of keys in a dictionary
    """
    return dict.keys()


CLASSES = {'Agaricus' : 0,'Amanita' : 1,'Boletus' : 2,'Cortinarius' : 3,'Entoloma' : 4,'Hygrocybe' : 5,
    'Lactarius' : 6,'Russula' : 7,'Suillus' : 8
                }

def get_label(key ,class_mush = CLASSES):
    """
    You can get the labeled name of the mushroom by giving 
    the index or the index by giving the key
    """
    if type(key) == str:
        return class_mush[key]
    elif type(key) == int:
        return list(CLASSES.keys())[list(CLASSES.values()).index(key)]


def upload_dir_labels(directory,mapping =CLASSES, k = 300):
    """
    give the list of photos from directory and the labels.
    k = number of pictures I want to take from each FOLDER. 
    """
    label = []
    dir= []
    list_of_mushrooms = getList(mapping)
    for f in list_of_mushrooms:
        pictures_dir = directory + f + '/'
        pictures = [pics for pics in listdir(pictures_dir) if isfile(join(pictures_dir , pics))]
        for idx, picture in enumerate(pictures[:k]): # take the first 300 pictures from each category
            label.append(get_label(f))  
            dir.append(pictures_dir + picture) 
        
    return dir, label

def image_reshape(dir, batch, new_size = (224,224),k = 300, gray =False):
    """
    give the list of photos with the correct pixel size you want to 
    downsize. If you want Gray then write True.
    """
    items_resized = []
    for idx, dir in enumerate(dir[(batch-1)*k:batch * k]):     
        item = cv2.imread(dir)
        if gray == True:
            item_gray = cv2.cvtColor(item, cv2.COLOR_BGR2GRAY)
            item_resized_gray = resize(item_gray, new_size)
            items_resized.append(item_resized_gray)
        else:
            item_resized_colored = resize(item, new_size)
            items_resized.append(item_resized_colored)
    with open(directory + "downsized/mushrooms_resized_part_"+ str(batch), "wb") as fp:   #Pickling
        pickle.dump(items_resized, fp)
    return items_resized


def downsize_in_batches(directories):
    """
    This function downsizes the images in batches of 300pics.
    directories = list of the directories of the images.
    """
    batch = 1
    for idx, dir in enumerate(directories):

        if idx % 300 == 0:
            print(batch)
            image_reshape(directories,batch)
            batch += 1 


def unpickle(directory , mapping):
    """
    This functions takes the data from a directory that have been
    downsized in banches ang gives back the X,y as arrays.
    """
    
    table_mushrooms_downsized = []
    directories, labels = upload_dir_labels(directory,mapping, k=300)
    for batch in range(1,10): # 10 is the number of batches
        with open(directory + "downsized/mushrooms_resized_part_"+ str(batch), "rb") as fp:   # Unpickling
            li = pickle.load(fp)
            table_mushrooms_downsized.append(li)

    # flatten the nested list into a 1D list
    mushrooms_downsized = list(itertools.chain(*table_mushrooms_downsized))
    if len(mushrooms_downsized) == len(labels):

        #Check if the number of pictures and labels are the same.
        X = np.asarray(mushrooms_downsized)
        y = np.asarray(labels)

    return X, y

def isqrt(n):
    """
    This returns the largest integer x for which x * x does not exceed n. 
    If you want to check if the result is exactly the square root, simply 
    perform the multiplication to check if n is a perfect square.
    """
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x
def plot_mushroom_coll(x = 25):
    """
    We can see a collection of random mushrooms with the labels
    x is the number of mushrooms you want to see and it will give 
    the appropriate number of n * n array.
    """
    for i in range(x):
        ran = random.randint(0,len(y)-1)
        plt.subplot(int(np.sqrt(x)), int(np.sqrt(x)), i+1)
        plt.imshow(X[ran], cmap='Greys')
        plt.axis('off')
        plt.text(0, 0, y[ran]) # displays y-values in each subplot


def pickle_(): #!!!!!!!!! Not ready function
    """
    This function pickles the data.
    """
    where ='clean_data/data_pool/train/'
    with open('X_train','wb') as f: pickle.dump(X_train, f)
    os.replace('train', where+ 'X_train')





def split_preprocessed_data(X ,y, test_size = 0.1, pp = False):
    """
    This function returns the train and validataion data 
    preprocessed if necessary: in a flattened 1_D form
    with normalized X and y per 255.
    Split images (75%/15%/10%) and save to temporary folders
    """
    if pp == True:
        x= X.reshape(X.shape[0],-1)
        x_norm = x.astype("float32") / 255 # Scale images to the [0, 1] range IF not scaled
        # one-hot-encode labels
        y_ohe = to_categorical(y)
        X_train, X_test, y_train, y_test = train_test_split(x_norm, y_ohe, test_size = test_size, random_state=42)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = test_size * 1.7, random_state=42) 
    else:
        x_norm = X.astype("float32") 
        # one-hot-encode labels
        y_ohe = to_categorical(y)
        X_train, X_test, y_train, y_test = train_test_split(x_norm, y_ohe, test_size = test_size, random_state=42)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = test_size * 1.7, random_state=42)  
    # file_name = 'data/Data/Mushrooms/'
    
    # pickle_(file_name + "train", X_train)
    # pickle_(file_name + "train", y_train)
    # pickle_(file_name + "test",  X_test)
    # pickle_(file_name + "test",  y_test)
    # pickle_(file_name + "valid", X_valid)
    # pickle_(file_name + "valid", y_valid)  
    return X_train, X_test, y_train, y_test, X_valid, y_valid


In [ ]:
# make a list of labels and directories
# Run to create batches labels etc
directory = 'data/Data/Mushrooms/' 
directories, labels = upload_dir_labels(directory, k=300)
downsize_in_batches()


- took 2 min and 36 sec for batches of 300 pictures per categories (300*300pixs)
- took 3 min and 12 sec for batches of 300 pictures per categories (224*224pixs)

In [ ]:
# Run to unpicle data and give back X,y
X ,y = unpickle(directory = 'data/Data/Mushrooms/' , mapping =CLASSES)

In [ ]:
plt.imshow(X[0],cmap='Greys')

In [ ]:
plot_mushroom_coll(x = 25)

- Prepare data to go inside the CNN models

- flatten data and normalize them to be in 0 to 1 intensity depending on the requirements of the model

In [ ]:
X_train, X_test, y_train, y_test, X_valid, y_valid = split_preprocessed_data(X,y, pp = False)

In [ ]:
plt.imshow(X_train[0],cmap='Greys')

- model RESNET_V2

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
import os
from tensorflow.keras.models import Model
from tensorflow import keras

In [ ]:
def model_ResNet50V2(CLASSES = CLASSES,learning_rate=0.001):
    """
    This function returns a ResNet50V2 model with the last
    layer removed.
    """
    K.clear_session() # Always clear the session!

    base_model = ResNet50V2(
        weights='imagenet', 
        pooling='avg',      # applies global average pooling to the output of the last conv layer (like a flattening)
        include_top=False,   # !!!!! we only want to have the base, not the final dense layers 
        input_shape=(224, 224, 3)  
    )
    base_model.trainable = False # To freeze the model
    # Start building on top of the model
    model = keras.Sequential() # defining a new model
    model.add(base_model) # adding in the pretrained model without the fully connected layer
    model.add(keras.layers.Dense(64, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(128, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(len(CLASSES), activation='softmax')) #!!! Final layer with a length of 2, and softmax activation 
    # have a look at the trainable and non-trainable params statistic
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

    return model

def fit_ResNet50V2(X_train, y_train,epochs = 5, batch_size =500,sav = False):
    """
    This function fits the model on the training data.
    """
    # observe the validation loss and stop when it does not improve after 3 iterations
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    history = model.fit(X_train, y_train, 
              epochs=epochs, 
              verbose=2,
              batch_size=batch_size, 
              callbacks=[callback],
              # use 30% of the data for validation
              validation_split=0.3)
    # save model
    if sav == True:
        model.save('models/ResNet50V2_RGB_300fot_SD.h5')
        
    return history

In [ ]:
model = model_ResNet50V2()

history = fit_ResNet50V2(X_train, y_train, epochs = 50, batch_size =500)

In [ ]:
model.evaluate(X_train,y_train)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])